In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import pathlib
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from glob import glob
from keras.utils import np_utils
from numpy.random import permutation
import os
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback


from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def read_load_resize(data):

    data_dir = pathlib.Path(data)
    print(data_dir)
    image_cnt = len(list(data_dir.glob('*/*.jpg')))
    print(image_cnt)

    position_img = {
        'c0':list(data_dir.glob('c0/*')),
        'c1':list(data_dir.glob('c1/*')),
        'c2':list(data_dir.glob('c2/*')),
        'c3':list(data_dir.glob('c3/*')),
        'c4':list(data_dir.glob('c4/*')),
        'c5':list(data_dir.glob('c5/*')),
        'c6':list(data_dir.glob('c6/*')),
        'c7':list(data_dir.glob('c7/*')),
        'c8':list(data_dir.glob('c8/*')),
        'c9':list(data_dir.glob('c9/*')),

    }
    position_labels = {
        'c0':0,
        'c1':1,
        'c2':2,
        'c3':3,
        'c4':4,
        'c5':5,
        'c6':6,
        'c7':7,
        'c8':8,
        'c9':9,

    }
    X, y = [], []

    for position_name, images in position_img.items():
        for image in images:
            img = cv2.imread(str(image))
            resized_img = cv2.resize(img,(224,224))
            X.append(resized_img)
            y.append(position_labels[position_name])
    return X,y

In [ ]:

pip install split-folders[full]

In [ ]:

import splitfolders
input_folder = '//content//drive//MyDrive//msid_dataset'
splitfolders.ratio(input_folder,output="Dataset",seed=42,ratio=(.7,.2,.1),group_prefix=None)

In [ ]:
IMAGE_SIZE = 224
CHANNELS = 3

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True
)

val_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True
)

test_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True
)

In [ ]:
train_data = '//content//Dataset//train'
val_data = '//content//Dataset//val'
x_train,y_train = read_load_resize(train_data)
x_val,y_val = read_load_resize(val_data)

In [ ]:
def normalize_data(X,y):
    X= np.array(X,dtype = np.uint8)
    y = np.array(y,dtype = np.uint8)
    X = X.transpose((0, 1, 2,3))
    X = X.astype('float16')
    mean_pixel = [80.857, 81.106, 82.928]
    X[:, :, :, 0] -= mean_pixel[0]
    X[:, :, :, 1] -= mean_pixel[1]
    X[:, :, :, 2] -= mean_pixel[2]
    y = np_utils.to_categorical(y, 10)
    perm = permutation(len(y))
    X = X[perm]
    y = y[perm]
    print('Train shape:', X.shape)
    print(X.shape[0], 'train samples')
    return X,y


In [ ]:
x_train ,y_train = normalize_data(x_train,y_train)
x_val,y_val = normalize_data(x_val,y_val)

In [ ]:

alpha = 1
model = Sequential([



    Conv2D(int(64 * alpha), (3, 3), strides=(1,1), padding='same',name= 'conv1'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),

    DepthwiseConv2D(3, strides=(2, 2), padding='same', name = 'd1'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(64 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p1'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),

    DepthwiseConv2D(3, strides=(2, 2), padding='same', name = 'd2'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(128 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p2'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),



    DepthwiseConv2D(3, strides=(2, 2), padding='same', name = 'd3'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(128 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p3'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),


    DepthwiseConv2D(3, strides=(1,1), padding='same', name = 'd4'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(256 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p4'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),


    DepthwiseConv2D(3, strides=(1,1), padding='same', name = 'd5'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(256 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p5'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),


    DepthwiseConv2D(3, strides=(2, 2), padding='same', name = 'd6'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(256 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p6'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),

    Dropout(0.05),


    DepthwiseConv2D(3, strides=(1,1), padding='same', name = 'd7'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(512 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p7'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),


    DepthwiseConv2D(3, strides=(1,1), padding='same', name = 'd8'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(512 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p8'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),


    DepthwiseConv2D(3, strides=(2, 2), padding='same', name = 'd9'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(512 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p9'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),

    Dropout(0.1),

    DepthwiseConv2D(3, strides=(1,1), padding='same', name = 'd10'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(512 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p10'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),


    DepthwiseConv2D(3, strides=(1,1), padding='same', name = 'd11'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(512 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p11'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),


    DepthwiseConv2D(3, strides=(2, 2), padding='same', name = 'd12'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(512 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p12'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),

    Dropout(0.2),

    DepthwiseConv2D(7, strides=(1,1), padding='same', name = 'd13'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Conv2D(int(512 * alpha), (1, 1), strides=(1, 1), padding='same',name = 'p13'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),

    Dropout(0.3),

    Conv2D(int(512 * alpha), (1,1), strides=(1,1), padding='same',name= 'conv2'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),

    GlobalMaxPooling2D(),

    layers.Flatten(),
    layers.Dense(10,activation = 'softmax')
])


In [ ]:
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model.compile(adam, loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.2,
            shear_range=0.2
            )

In [ ]:
datagen.fit(x_train)

In [ ]:
nb_epoch = 55

In [ ]:
hist=model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                steps_per_epoch=len(x_train)*0.8// 32, epochs=nb_epoch,
        verbose=1, validation_data=(x_val, y_val),
                         validation_steps = len(x_val)*0.8//32,
                         )


In [ ]:
test_data = '/home/sggs/dataset/test'
x_test,y_test = read_load_resize(test_data)
x_test,y_test = normalize_data(x_test,y_test)

In [ ]:
loss, acc = model.evaluate(x_test,y_test, verbose=2)

In [ ]:
model.save_weights('./checkpoints/my_checkpoint7')

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model7')

model.save('saved_model/my_model7.h5')

In [ ]:
import matplotlib
import matplotlib.pyplot as plot

In [ ]:
plot.plot(hist.history['accuracy'])
plot.plot(hist.history['val_accuracy'])
plot.title('model accuracy')
plot.ylabel('accuracy')
plot.xlabel('epoch')
plot.legend(['train', 'val'], loc='lower right')

In [ ]:
plot.plot(hist.history['loss'])
plot.plot(hist.history['val_loss'])
plot.title('model loss')
plot.ylabel('loss')
plot.xlabel('epoch')
plot.legend(['train', 'test'], loc='upper right')

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (224, 224, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image


In [ ]:

img  = '/home/sggs/dataset/test/c5/img_578.jpg'
image = load(img)
print(hist.model.predict(image))

In [ ]:
predictions = model.predict_classes(X_test)
correct_predictions = np.nonzero(predictions == y_test)[0]
incorrect_predictions = np.nonzero(predictions != y_test)[0]
print(len(correct_predictions)," classified correctly")
print(len(incorrect_predictions)," classified incorrectly")